+ 혁펜하임의 ATT강의를 듣고 참고하여 정리한 내용.

# ViT : Vision Transformer

## Patch Embedding

<img src="https://drive.google.com/uc?id=15xs0WZLRNX1K5GTZb13lWQg2gYvJqD8C" height=200>

+ 트랜스포머를 이미지 분류에 그대로 적용.

+ 전체 이미지를 패치로 쪼개서 사용.
    + 각 이미지 = 문장
    + 각 패치 = 단어

와 같이 생각할 수 있다.

즉, 패치단위로 나누고, 각 픽셀값을 한줄로 벡터화하면 입력벡터가 된다.

## Inductive bias trade off

<img src="https://drive.google.com/uc?id=1mncQLQDfSZk1CetZ0bzkTNWABL1FZX2L" height=200> <img src="https://drive.google.com/uc?id=1sJ3WoumoLx0H8LTD7UMjSaDMVo8fR6OZ" height=200>

+ CNN의 경우엔 3x3 두번 통과하면 한 픽셀당 5x5라는 receptive field를 갖게됨.
    + 이는, 레이어를 통과할 때마다 가까운 곳의 세밀한 곳부터 보고 점점 전체적으로 정보를 담도록 하는 것.
    + Inductive bias 큼.
    + 적은 데이터셋에 좋음.

+ ViT의 경우엔 Transformer encoder를 통과하면서 픽셀끼리 중요한 애들을 알아서 attention하도록 함.
    + 언제, 어디서 어떤 receptive field를 가질지 모델이 알아서 알아내도록 함.(layer 초반? 얼마나 크게? 어떤 모양?)
    + Inductive bias 작음.
    + 큰 데이터셋에 좋음.

## ViT architecture

<img src="https://drive.google.com/uc?id=1Blh7Jb6qAb49xQ8T_o8jsFQ2xFBMsnY_" height=400>

+ Transformer Encoder인 이유는 masked-attention을 하지 않으므로.

1. patch 단위로 쪼갠다.
2. 각 패치를 flatten하여 Linear 통과
    + 1, 2를 nn.Conv2d(3, dim, kernel_size=14, stride=14)통과시키고 (N, dim, H, W)
    + rearrange로 구현. (N, HxW, dim)
3. 위치정보를 위해 positional embedding vector 더해줌
4. CLS 토큰을 추가.(learnable parameter로, 0벡터 초기화)
5. 최종 CLS 의 embedding vector를 MLP 통과시켜 분류.

## ViT mathematical representation

<img src="https://drive.google.com/uc?id=1X2LSafGci9MW5bvnHDBnwgs4BdZG7W14" height=150>

## Experiments

<img src="https://drive.google.com/uc?id=1mN4zsYRNUjI7VbL3ci_jhPn0gp1oYo5y" height=200>

기존모델 모든 경우에서 다 앞섰다. (Noisy Student라는 것도 JFT가지고 pretrain된 모델임.)

<img src="https://drive.google.com/uc?id=1BykVQszoTz1582Uq4OIdYnnKlOCAa_AB" height=400>

+ BiT : CNN 모델. 즉, 적은 데이터셋일수록 CNN이 더 잘한다. __inductive bias때문!__

<img src="https://drive.google.com/uc?id=1X-E0OJr9zbEmO8huZVFNaoDNPXrBJPdf" height=300>

+ 여러 방법측면에서 연산수가 높으면, 성능이 골고루 향상됨.
+ __scalable한 모델이다!__

## Insight experiments

<img src="https://drive.google.com/uc?id=1zoZtp2As5_zcGh1vChOXZQlFVapNz0mg" height=300>

+ 처음에 linear embedding할 때 사용한 filter를 시각화하였다..
+ 구현할 때 nn.Conv2d(3, 768, kernel_size=32, stride=32)로 구현하므로, 각각은 3x32x32의 형태를 띄고있을 것임.
+ 이를 PCA해서 상위 28개의 filter를 뽑아보았다.
+ 선, 격자를 봄. (CNN을 시각화 했을 때와 유사.)

<img src="https://drive.google.com/uc?id=1HFmKaULzqPLJDLDz5gm3Q0-S6kmsDLtD" height=300>

+ Position embedding vector를 학습 파라미터로 사용하기 때문에 유의미한 실험.
+ 각각의 $E_{pos}$간에 cos similarity를 계산한 것
+ 7x7개의 position이 있을 것이고, 이걸 각각 서로 다른애들이랑 계산해놓은걸 한번에 보여준 것이다.
+ __가까울 수록 유사도가 높다__
+ __같은 행, 열일수록 유사도가 높다__ <- 이부분이 좀 신기함. 이미지가 똑바로 찍혀있는게 대부분이라 그런가 flip 등의 augmentation 때문인가 싶기도함.

<img src="https://drive.google.com/uc?id=1sJ3WoumoLx0H8LTD7UMjSaDMVo8fR6OZ" height=350>

+ Attention weights를 이용해서, 각 패치간 거리(pixel단위)를 weighted mean해본 것.

<img src="https://drive.google.com/uc?id=1AdYJAvwPzXCQt9QuKdF5zO3BJ-vjiQWC" height=300>

__초반 layer부터 멀리 있는 패치정보를 보고있다.__  
CNN의 receptive field가 layer를 통과할 수록 넓어지는 것과 비교해볼 수 있다.